In [4]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [6]:
# Configurations

import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_remote_path_root = pathlib.Path('/webdav')
conf_remote_path_ahn = pathlib.Path('/webdav/ahn')
conf_remote_path_retiled = pathlib.Path('/webdav/retiled/')
conf_remote_path_targets = pathlib.Path('/webdav/targets')
conf_local_tmp = pathlib.Path('/tmp')

param_hostname = ''
param_login = ''
param_password = ''

param_feature_name = 'perc_95_normalized_height'
param_validate_precision = '0.00001'
param_tile_mesh_size = '10.'
param_filter_type= 'select_equal'
param_attribute = 'raw_classification'
param_min_x = '-113107.81'
param_max_x = '398892.19'
param_min_y = '214783.87'
param_max_y = '726783.87'
param_n_tiles_side = '512'
param_apply_filter_value = '1'

conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}

## Fetching Laz Files from remote WebDAV

In [7]:
# Fetch Laz Files 19-04-22
laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_ahn.as_posix())
             if f.lower().endswith('.laz')]

## Retiling of big files into smaller tiles

In [11]:
# Retiling 


from IPython.display import display, HTML   

remote_path_retiled = str(conf_remote_path_retiled)

grid_retile = {
    'min_x': float(param_min_x),
    'max_x': float(param_max_x),
    'min_y': float(param_min_y),
    'max_y': float(param_max_y),
    'n_tiles_side': int(param_n_tiles_side)
}


retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_ahn.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}


# try:
#     get_ipython
#     file = laz_files[0]
# except:
#     file = laz_files
    
file = laz_files
    
retiler = Retiler(file,label=file).config(retiling_input).setup_webdav_client(conf_wd_opts)
retiler_output = retiler.run()

2022-04-14 16:41:09,727 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/C_18HZ2.LAZ_input
2022-04-14 16:41:09,728 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/C_18HZ2.LAZ_output
2022-04-14 16:41:09,732 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/ahn/C_18HZ2.LAZ ...
2022-04-14 16:41:10,888 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-14 16:41:10,889 -                        laserfarm.retiler -       INFO - Setting up the target grid
2022-04-14 16:41:10,892 -                        laserfarm.retiler -       INFO - Splitting file /tmp/C_18HZ2.LAZ_input/C_18HZ2.LAZ with PDAL ...
2022-04-14 16:41:11,351 -                        laserfarm.retiler -       INFO - ... splitting completed.
2022-04-14 16:41:11,352 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2022-04-14 16:41:11,354 -                

## Fetching retilied files (tiles) from remote WebDAV

In [12]:
# Fetch Tiles 
remote_path_retiled

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

## Extract features from tiles

In [15]:
# Feature Extraction


# try:
#     get_ipython
#     t = tiles[0]
# except:
    # t = tiles    
    
t = tiles
    

features = [param_feature_name]

tile_mesh_size = float(param_tile_mesh_size)

grid_feature = {
    'min_x': float(param_min_x),
    'max_x': float(param_max_x),
    'min_y': float(param_min_y),
    'max_y': float(param_max_y),
    'n_tiles_side': int(param_n_tiles_side)
}

feature_extraction_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_retiled.as_posix(),
    'load': {'attributes': [param_attribute]},
    'normalize': 1,
    'apply_filter': {
        'filter_type': param_filter_type, 
        'attribute': param_attribute,
        'value': [int(param_apply_filter_value)]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        'validate_precision': float(param_validate_precision),
        **grid_feature
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'pushremote': conf_remote_path_targets.as_posix(),
#     'cleanlocalfs': {}
}



    
    
    
idx = (t.split('_')[1:])
    
processing = DataProcessing(t, tile_index=idx,label=t).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
processing.run()

2022-04-14 16:42:17,149 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/tile_278_391_input
2022-04-14 16:42:17,151 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/tile_278_391_output
2022-04-14 16:42:17,153 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/retiled/tile_278_391 ...
2022-04-14 16:42:19,761 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-14 16:42:19,765 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2022-04-14 16:42:19,766 -                laserfarm.data_processing -       INFO - ... loading /tmp/tile_278_391_input/tile_278_391/C_01GN2_1.LAZ
2022-04-14 16:42:19,823 -                          pylas.lasreader -      ERROR - lazrs failed to decompress points: lazrs is not installed
2022-04-14 16:42:23,983 -                laserfarm.data_processing -       INFO - ... loading completed.
2022-04-14 

In [19]:
# GeoTIFF Export

feature = features

remote_path_geotiffs = conf_remote_path_ahn.parent / 'geotiffs'

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=param_feature_name, bands=param_feature_name,label=param_feature_name).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()

2022-04-15 08:54:22,739 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/perc_95_normalized_height_input
2022-04-15 08:54:22,742 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/perc_95_normalized_height_output
2022-04-15 08:54:22,744 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/targets/perc_95_normalized_height ...
2022-04-15 08:54:48,012 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-15 08:54:48,014 -                 laserfarm.geotiff_writer -       INFO - 27 PLY files found
2022-04-15 08:54:48,067 -                 laserfarm.geotiff_writer -       INFO - No. of points per file: 10000
2022-04-15 08:54:48,070 -                 laserfarm.geotiff_writer -       INFO - Resolution: (10.0m x 10.0m)
2022-04-15 08:54:48,071 -                 laserfarm.geotiff_writer -       INFO - Splitting data into (1x1) sub-regions
2022-04-15 08:54:48,072 -     

In [ ]:
print(remote_path_geotiffs)